# Q 4 B. Approximate Stream Regression

## Generator Function 
### Generate_X for Independent variables
### Generate_y for Dependent variables

In [0]:
def generate_X(X): #Generator function for sending data in the multiple of 100 (i.e 1-100,101-200,...)
    for i in range(4): # Here i have consider 500 data point
        yield X[i*100:(i+1)*100,:]
def generate_y(y): #Generator function for sending data in the multiple of 100 (i.e 1-100,101-200,...)
    for i in range(4): # Here i have consider 500 data point
        yield y[i*100:(i+1)*100]

# ASR works on this equation
![](m1.PNG)

In [2]:
# Required Libraries
import numpy as np
from sklearn.datasets import load_boston
from numpy.linalg import pinv,inv,LinAlgError
from sklearn import metrics

# hyperparameters
alpha = 0.8
theta_list=[]
count=0

# loading boston dataset
X,y = load_boston(return_X_y=True)
dummy1 = np.ones((X[400:506].shape[0],1)) 
X_test = np.c_[dummy1,X[400:506]] # (106,13) data points for testing
y_test = y[400:506] # 106 data points for testing


# creating instances of generatior function
A = generate_X(X) # For X_train 
B = generate_y(y) # For y_train   
    
try:
    while True:
        X_train_temp1 = next(A) # yeilding the next X_train
        dummy = np.ones((X_train_temp1.shape[0],1)) 
        X_train = np.c_[dummy,X_train_temp1] #Appending ones column in the front
        y_train = next(B) # yeilding the next y_train
        
        theta_curr = np.zeros((X_train.shape[1],1)) #Initializing parameters
        temp1 = np.zeros((X_train.shape[1],1)) #Dummy variable
        temp2 = np.zeros((X_train.shape[1],1)) #Dummy variable
        theta_final = np.zeros((X_train.shape[1],1)) #For final Theta which is calculated based on ASR formula

        XTX = np.dot(X_train.T,X_train)  # Computing XTX
        try:
            XTXi = inv(XTX) # Computing XTX inverse
        except:
            XTXi = pinv(XTX) # Computing XTX pusedo inverse

        XTy = np.dot(X_train.T,y_train) # Computing XTy
        theta_curr = np.dot(XTXi,XTy.T) # parameter based on current window 
        if count<1: # For first iteration
            theta_list.append(theta_curr) #Appending first value of theta
            count=count+1
        else: # Logic of ASR formula (i.e mentioned in the picture above)
            for i in range(len(theta_curr)):
                temp1[i] = theta_curr[i]*alpha
            for i in range(len(theta_curr)):
                temp2[i] = theta_list[count-1][i]*(1-alpha)
            for i in range(len(theta_curr)):
                theta_final[i] = temp1[i] + temp2[i]
            theta_list.append(theta_final)
            count=count+1
        
        print(10*"-") # Doing testing for the current parameter
        prediction = np.dot(X_test,theta_list[count-1]) # theta_list[19] theta estimated based in 1-400 data points 
        print("MAE: ",metrics.mean_absolute_error(y_true=y_test,y_pred=prediction))
        print("MSE: ",metrics.mean_squared_error(y_true=y_test,y_pred=prediction))
        print(10*"-")
        
except Exception as e:
    print("Train Data Set is consumed")

----------
MAE:  36.81238430315513
MSE:  3181.040342667544
----------
----------
MAE:  9.667787600522994
MSE:  219.37747462330216
----------
----------
MAE:  7.61930367189083
MSE:  104.72756738538637
----------
----------
MAE:  4.754782014174462
MSE:  34.3296687091723
----------
Train Data Set is consumed
